In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist
from keras import backend as K
import math
import keras.callbacks

print(keras.__version__)

Using TensorFlow backend.


2.0.5


In [16]:
from keras.layers import Input, Dense, Embedding, LSTM
from keras.models import Model

In [17]:
# This returns a tensor
main_input = Input(shape=(100,), dtype='int32', name='main_input')
x = Embedding(output_dim=512, input_dim=10000, input_length=100)(main_input)
lstm_out = LSTM(32)(x)

auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(lstm_out)

auxiliary_input = Input(shape=(5,), name='aux_input')
x = keras.layers.concatenate([lstm_out, auxiliary_input])

# We stack a deep densely-connected network on top
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# And finally we add the main logistic regression layer
main_output = Dense(1, activation='sigmoid', name='main_output')(x)
model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output, auxiliary_output])

In [18]:
model.summary()
print model.output_shape

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
main_input (InputLayer)          (None, 100)           0                                            
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 100, 512)      5120000     main_input[0][0]                 
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 32)            69760       embedding_2[0][0]                
____________________________________________________________________________________________________
aux_input (InputLayer)           (None, 5)             0                                            
___________________________________________________________________________________________

In [19]:
yaml_string = model.to_yaml()
open('new_model_architecture.yml', 'w').write(yaml_string)

In [21]:
from keras.layers.core import Lambda

In [22]:
model = None
model = Sequential()
 
model.add(Conv2D(32, (3, 3), data_format="channels_first", activation='tanh', input_shape=(1,28,28)))
model.add(Lambda(lambda x: x ** 2))


In [23]:
model.summary()
print model.output_shape

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 32, 26, 26)        320       
_________________________________________________________________
lambda_1 (Lambda)            (None, 32, 26, 26)        0         
Total params: 320
Trainable params: 320
Non-trainable params: 0
_________________________________________________________________
(None, 32, 26, 26)


In [24]:
yaml_string = model.to_yaml()
open('new_model_architecture.yml', 'w').write(yaml_string)

In [26]:
from keras.engine.topology import Layer

In [34]:
class MyLayer(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(MyLayer, self).build(input_shape)  # Be sure to call this somewhere!

    def call(self, x):
        return K.dot(x, self.kernel)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [39]:
model = None
model = Sequential()
l = MyLayer(32, input_shape=(28,28))
model.add(l)
model.add(Lambda(lambda x: x ** 2))

In [40]:
model.summary()
print model.output_shape

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
my_layer_4 (MyLayer)         (None, 32)                896       
_________________________________________________________________
lambda_2 (Lambda)            (None, 32)                0         
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________
(None, 32)


In [41]:
yaml_string = model.to_yaml()
open('mylayer_model_architecture.yml', 'w').write(yaml_string)